In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy.stats import chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


# 加载数据集
dt = pd.read_csv('SurveyData.csv')
income = dt["IncomeLevel"]
converted_sub = dt["Non-Subscribe"]
sub = [1-x for x in converted_sub]
time = dt["ListeningTime"]


In [ ]:
# 首先分别对两个变量是否与订阅存在显著关系进行看房检验
chi2, p, dof, expected = chi2_contingency([pd.crosstab(income, sub)])
print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
if p < 0.05:
    print("在显著水平为0.05的情况下，拒绝原假设，收入与订阅存在显著关系。")
else:
    print("在显著水平为0.05的情况下，不能拒绝原假设，收入与订阅不存在显著关系。")

chi2, p, dof, expected = chi2_contingency([pd.crosstab(time, sub)])
print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
if p < 0.05:
    print("在显著水平为0.05的情况下，拒绝原假设，时间与订阅存在显著关系。")
else:
    print("在显著水平为0.05的情况下，不能拒绝原假设，时间与订阅不存在显著关系。")

由卡方检验可知，两个变量都与是否订阅存在显著关系。

In [ ]:
# 接着探究两个自变量之间即收入与听歌时长之间是否存在显著关系
# 因为数据将收入分为了区间，而听歌时长也同样分为了区间。因此将两者都视作分类变量。
# 则H0: 收入与听歌时长之间不存在显著关系
# H1: 收入与听歌时长之间存在显著关系

# 创建列联表并添加总计
contingency_table1 = pd.crosstab(income, time, rownames=['IncomeLevel'], colnames=['Listening Time'], margins=True)
# 列联表
print(contingency_table1)

# 进行卡方检验
chi2, p, dof, expected = chi2_contingency([pd.crosstab(income, time)])
print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
if p < 0.05:
    print("在显著水平为0.05的情况下，拒绝原假设，存在显著关系。")
else:
    print("在显著水平为0.05的情况下，不能拒绝原假设，不存在显著关系。")

依据卡方检验，我们认为两者之间不存在显著关系。即两者之间是独立的。

所以接下来我们进行对两者与是否订阅之间的关系的具体分析。
因为因变量是个分类变量，所以我们不能采用线性回归的方法进行分析。考虑到是否订阅是个二分类的分类变量，逻辑回归的方法在此处可能更为合适。

In [ ]:
#准备填充数据用的pipeline
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
pipeline = Pipeline([
    ('imputer', imputer),
    ('classifier', LogisticRegression())
])

# 准备数据
X = pd.DataFrame({'IncomeLevel': income, 'ListeningTime': time})  
y = np.array(sub)  

# 确保X和y中没有缺失值
X = X.dropna()
y = y[X.index]  

#进行数据集划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用管道训练模型
pipeline.fit(X_train, y_train)

# 预测
y_pred = pipeline.predict(X_test)

# 评估模型
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# 获取训练后的逻辑回归模型
trained_classifier = pipeline.named_steps['classifier']

# 查看系数
print("对应系数:", trained_classifier.coef_)

# 查看截距
print("截距:", trained_classifier.intercept_)

In [ ]:
plt.rcParams['font.sans-serif'] = ['Hiragino Sans GB']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 300

# 获取逻辑回归模型的系数和截距
coef = trained_classifier.coef_[0]  # 系数
intercept = trained_classifier.intercept_  # 截距

# 创建网格，用于计算每个点的预测概率
x_min, x_max = X['IncomeLevel'].min() - 1, X['IncomeLevel'].max() + 1
y_min, y_max = X['ListeningTime'].min() - 1, X['ListeningTime'].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))

# 计算网格点的预测概率
grid = np.c_[xx.ravel(), yy.ravel()]
probs = trained_classifier.predict_proba(grid)[:, 1].reshape(xx.shape)

# 绘制决策边界
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, probs, alpha=0.7, cmap="RdBu_r")
plt.colorbar(label='Probability of Subscription')

# 添加样本点
plt.scatter(X['IncomeLevel'], X['ListeningTime'], c=y, edgecolor='k', s=50,
            cmap=ListedColormap(['blue', 'red']), label="Subscription")
plt.legend(["0 (Not Subscribed)", "1 (Subscribed)"], loc="upper right")

# 设置标题和坐标轴
plt.title("Logistic Regression Decision Boundary")
plt.xlabel("Income Level")
plt.ylabel("Listening Time Per Day")
plt.show()


由之前的结论，听歌时长和收入都对订阅有着显著影响。
而由逻辑回归的系数来看，听歌时长对是否订阅的影响比收入的影响更大。